In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Проблема

В статистике часто приходится считать выборочное среднее, т.е. по данной выборке значений $x_k$, $k=1..N$, нужно вычислить
$$\bar x=\frac1N\sum_{k=1}^N x_k.$$
С точки зрения математики не имеет значения, как считать указанную сумму, так как результат сложения всегда будет один и тот же.
Однако при вычислениях с плавающей запятой ответ будет зависеть от порядка выполнения операций, хотя бы потому, что сложения чисел с плавающей запятой не ассоциативно.
Но будет ли зависеть точность вычислений от порядка операций?
Давайте это проверим.

Сконструируем выборку таким образом, что сумма всех элементов равна $1$, и порядок элементов меняется в широком диапазоне.
Для этого разобьем единицу на $K$ частей, и $k$-ую часть разобьем на $7^k$ равных значений.
Полученные элементы перемешаем.

In [2]:
def exact_sum(K):
    """Точное значение суммы всех элементов."""
    return 1.

def samples(K, base=7):
    """"Элементы выборки"."""
    # создаем K частей из base^k одинаковых значений
    parts=[np.full((base**k,), float(base)**(-k)/K) for k in range(0, K)] 
    # создаем выборку объединяя части
    samples=np.concatenate(parts) 
    # перемешиваем элементы выборки и возвращаем
    return np.random.permutation(samples)

def direct_sum(x):
    """Последовательная сумма всех элементов вектора x"""
    s=0.
    for e in x: 
        s+=e
    return s

Создадим выборку из значений, отличающихся на 6 порядков, и просуммируем элементы выборки.

In [3]:
K=7 # число слагаемых
x=samples(K) # сохраняем выборку в массив
print("Число элементов:", len(x))
print("Самое маленькое и большое значения:", np.min(x), np.max(x))

exact_sum_for_x=exact_sum(K) # значение суммы с близкой к машинной погрешностью
direct_sum_for_x=direct_sum(x) # сумма всех элементов по порядку

def relative_error(x0, x):
    """Погрешность x при точном значении x0"""
    return np.abs(x0-x)/np.abs(x)

print("Погрешность прямого суммирования:", relative_error(exact_sum_for_x, direct_sum_for_x))

Число элементов: 137257
Самое маленькое и большое значения: 1.2142656789e-06 0.142857142857
Погрешность прямого суммирования: 1.09323661235e-12


Попробуем теперь просуммировать элементы в порядке возрастания.

In [4]:
sorted_x=x[np.argsort(x)]
sorted_sum_for_x=direct_sum(sorted_x)
print("Погрешность суммирования по возрастанию:", relative_error(exact_sum_for_x, sorted_sum_for_x))

Погрешность суммирования по возрастанию: 3.09974268475e-13


Попробуем просуммировать в порядке убывания.

In [5]:
sorted_x=x[np.argsort(x)[::-1]]
sorted_sum_for_x=direct_sum(sorted_x)
print("Погрешность суммирования по убыванию:", relative_error(exact_sum_for_x, sorted_sum_for_x))

Погрешность суммирования по убыванию: 1.6949774917e-12


Таким образом погрешность результата зависит от порядка суммирования. 
Как можно объяснить этот эффект?

На практике суммирование предпочтительно проводить не наивным способом, а компенсационным суммированием (см. [алгоритм Кэхэна](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%9A%D1%8D%D1%85%D1%8D%D0%BD%D0%B0).

In [6]:
def Kahan_sum(x):
    s=0.0 # частичная сумма
    c=0.0 # сумма погрешностей
    for i in x:
        y=i-c      # первоначально y равно следующему элементу последовательности
        t=s+y      # сумма s может быть велика, поэтому младшие биты y будут потеряны
        c=(t-s)-y  # (t-s) отбрасывает старшие биты, вычитание y восстанавливает младшие биты
        s=t        # новое значение старших битов суммы
    return s

Kahan_sum_for_x=Kahan_sum(x) # сумма всех элементов по порядку
print("Погрешность суммирования по Кэхэну:", relative_error(exact_sum_for_x, Kahan_sum_for_x))

Погрешность суммирования по Кэхэну: 0.0


Что позволяет алгоритму Кэхэна получать результат с высокой точностью?

# Задания

1. Объясните различие в погрешностях при различных порядках суммирования.
3. Почему алгорит Кэхэна имеет значительно лучшую точность, чем последовательное суммирование?
3. Получим ли мы те же значения погрешностей, если будем суммировать последовательность со слагаемыми разных знаков? Проверьте на следующей последовательности: 
$$x_k=\sin k.$$
4. Что произойдет с погрешностью, если элементы выборки с разными знаками упорядочить по возрастанию? По возврастанию абсолютной величины? Проверьте экспериментально.

# Подсказка

Сумма первых $N$ элементов последовательности из задания 4 может быть найдена явна:
$$\sum_{k=1}^N\sin k=\frac12\bigg(\sin n+\mathrm{ctg}\frac12\cos n+\mathrm{ctg}\frac12\bigg).$$